In [1]:
print("ok")

ok


In [2]:
%pwd


'c:\\Users\\User\\Desktop\\flask medi chatbot\\research'

In [3]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\User\\Desktop\\flask medi chatbot'

In [4]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
#extract data from the pdf
def load_pdf_file(data):
    loader= DirectoryLoader(data, 
                            glob= "*.pdf",
                            loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf_file(data="Data/")

In [7]:
extracted_data

[Document(metadata={'producer': 'Acrobat Distiller 9.4.5 (Macintosh)', 'creator': 'Adobe InDesign CS4 (6.0.6)', 'creationdate': '2013-07-24T20:36:08+00:00', 'author': '', 'ebx_publisher': 'Dorling Kindersley Limited', 'enhanced': 'By PDF Enhancer 3.5.6359/Win', 'moddate': '2014-03-01T09:48:57+10:00', 'spdf': '1127', 'title': 'The Psychology Book', 'source': 'Data\\psychology_explained.pdf', 'total_pages': 354, 'page': 0, 'page_label': 'Cover'}, page_content=''),
 Document(metadata={'producer': 'Acrobat Distiller 9.4.5 (Macintosh)', 'creator': 'Adobe InDesign CS4 (6.0.6)', 'creationdate': '2013-07-24T20:36:08+00:00', 'author': '', 'ebx_publisher': 'Dorling Kindersley Limited', 'enhanced': 'By PDF Enhancer 3.5.6359/Win', 'moddate': '2014-03-01T09:48:57+10:00', 'spdf': '1127', 'title': 'The Psychology Book', 'source': 'Data\\psychology_explained.pdf', 'total_pages': 354, 'page': 1, 'page_label': 'C'}, page_content=''),
 Document(metadata={'producer': 'Acrobat Distiller 9.4.5 (Macintosh)',

In [8]:
#split the data into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks_ = text_splitter.split_documents(extracted_data)

    return text_chunks_

In [9]:
text_chunks = text_split(extracted_data)
print("length of text chunks: ", len(text_chunks))

length of text chunks:  1975


In [10]:
text_chunks

[Document(metadata={'producer': 'Acrobat Distiller 9.4.5 (Macintosh)', 'creator': 'Adobe InDesign CS4 (6.0.6)', 'creationdate': '2013-07-24T20:36:08+00:00', 'author': '', 'ebx_publisher': 'Dorling Kindersley Limited', 'enhanced': 'By PDF Enhancer 3.5.6359/Win', 'moddate': '2014-03-01T09:48:57+10:00', 'spdf': '1127', 'title': 'The Psychology Book', 'source': 'Data\\psychology_explained.pdf', 'total_pages': 354, 'page': 2, 'page_label': '1'}, page_content='THE\nPSYCHOLOGY\nBOOK'),
 Document(metadata={'producer': 'Acrobat Distiller 9.4.5 (Macintosh)', 'creator': 'Adobe InDesign CS4 (6.0.6)', 'creationdate': '2013-07-24T20:36:08+00:00', 'author': '', 'ebx_publisher': 'Dorling Kindersley Limited', 'enhanced': 'By PDF Enhancer 3.5.6359/Win', 'moddate': '2014-03-01T09:48:57+10:00', 'spdf': '1127', 'title': 'The Psychology Book', 'source': 'Data\\psychology_explained.pdf', 'total_pages': 354, 'page': 4, 'page_label': '3'}, page_content='THE\nPSYCHOLOGY\nBOOK'),
 Document(metadata={'producer': 

In [11]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [12]:
# download the embeddings modelfrom huggingface
def download_Hugging_face_embeddings():
    embeddings= HuggingFaceBgeEmbeddings(model_name="all-mpnet-base-v2",)
    return embeddings

In [13]:
import sentence_transformers



c:\Users\User\miniconda3\envs\medibot-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
embeddings= download_Hugging_face_embeddings()

C:\Users\User\AppData\Local\Temp\ipykernel_15200\438362739.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceBgeEmbeddings(model_name="all-mpnet-base-v2",)


In [15]:
query_result = embeddings.embed_query("hello world")
print("length of query result: ", len(query_result))

length of query result:  768


In [16]:
#query_result

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
import os

pinecone_api_key=os.environ.get("pcsk_79T3NW_4cJFZDRwRiyW9bNMzQL9qbHXpoXfXiFNiiuT9hY2uWTdZWxz4tFC5AvudpsrqY9")

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone

import os

pc = Pinecone(api_key="pcsk_79T3NW_4cJFZDRwRiyW9bNMzQL9qbHXpoXfXiFNiiuT9hY2uWTdZWxz4tFC5AvudpsrqY9")


index_name = "medibot"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

c:\Users\User\miniconda3\envs\medibot-env\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at db_data_2025-04.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [20]:
import os
import pinecone

pinecone_api_key= "pcsk_79T3NW_4cJFZDRwRiyW9bNMzQL9qbHXpoXfXiFNiiuT9hY2uWTdZWxz4tFC5AvudpsrqY9"
os.environ["PINECONE_API_KEY"] = ""  # <-- Set key as string

pinecone_api_key = os.environ.get("PINECONE_API_KEY")  # Now safely fetch

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Step 1: Init Pinecone client
pc = Pinecone(api_key="pcsk_79T3NW_4cJFZDRwRiyW9bNMzQL9qbHXpoXfXiFNiiuT9hY2uWTdZWxz4tFC5AvudpsrqY9")  # ✅ Your real key

# Step 2: Index name & creation
index_name = "medibot"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # ✅ Match your embedding model
        spec=ServerlessSpec(cloud="aws", region="us-west-2")  # ✅ Match your dashboard
    )

# Step 3: Get index object (most important!)
index = pc.Index(index_name)

# Step 4: Use with Langchain
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index=index  # ✅ This is required now instead of api_key/env
)


PineconeConfigurationError: You haven't specified an API key. Please either set the PINECONE_API_KEY environment variable or pass the 'api_key' keyword argument to the Pinecone client constructor.